# PIPELINE ML PARA PREDICCIÓN DE FÚTBOL

## Proyecto: Machine Learning para Premier League

**Objetivo**: Crear pipeline completo de ML para predecir resultados de partidos de fútbol.

**Datos**: Premier League 2017-2025 (3,035+ partidos, 31 equipos)

**Modelos**: 
- Elo Baseline
- Poisson Baseline  
- XGBoost Avanzado

**Features**: 103+ características automáticas (Elo, forma, estadísticas avanzadas)


# 1. Importación de Librerías Necesarias

In [23]:
# ========================================================================================
# IMPORTACIONES Y CONFIGURACIÓN
# ========================================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar visualización
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Librerías importadas exitosamente")
print("Configuración de visualización aplicada")
print("Sistema listo para pipeline ML")

# Configuración de datos
DATA_DIR = r"c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_Mining\eda_outputsMatchesPremierLeague"
print(f"Directorio de datos: {DATA_DIR}")

print("\nPIPELINE ML INICIADO!")
print("=" * 50)

Librerías importadas exitosamente
Configuración de visualización aplicada
Sistema listo para pipeline ML
Directorio de datos: c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_Mining\eda_outputsMatchesPremierLeague

PIPELINE ML INICIADO!


# 2.  Ejecución del Pipeline ETL Completo

Utilizamos nuestro pipeline ETL personalizado para extraer datos de PostgreSQL, realizar feature engineering avanzado y preparar los datos para Machine Learning.

In [36]:
# ========================================================================================
# SECCIÓN 2: CARGA OPTIMIZADA DE DATOS - match_data_cleaned.csv
# ========================================================================================

import pandas as pd
import numpy as np

print("CARGA MEJORADA DE DATOS CON ESTADÍSTICAS AVANZADAS")
print("=" * 60)

# ========================================================================================
# CARGAR DIRECTAMENTE match_data_cleaned.csv CON ESTADÍSTICAS AVANZADAS
# ========================================================================================

# Ruta al archivo con estadísticas completas
csv_path = r"c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_Mining\eda_outputsMatchesPremierLeague\match_data_cleaned.csv"

print("Cargando datos desde match_data_cleaned.csv...")
print("Este archivo contiene estadísticas más avanzadas y completas")

try:
    # Cargar datos
    matches_df = pd.read_csv(csv_path)
    
    # Convertir fecha
    matches_df['date_game'] = pd.to_datetime(matches_df['date_game'])
    
    print(f" CARGA EXITOSA!")
    print(f"   Registros totales: {len(matches_df):,}")
    print(f"   Rango temporal: {matches_df['date_game'].min()} a {matches_df['date_game'].max()}")
    print(f"   Partidos únicos: {matches_df['match_id'].nunique():,}")
    print(f"   Equipos únicos: {matches_df['team_name'].nunique():,}")
    print(f"   Temporadas: {matches_df['season_id'].nunique():,}")
    
    # Información de columnas
    print(f"   Total columnas: {len(matches_df.columns):,}")
    
    # Mostrar primeras columnas para verificar datos
    print(f"\nPRIMERAS 10 COLUMNAS:")
    for i, col in enumerate(matches_df.columns[:10]):
        print(f"   {i+1:2d}. {col}")
    
    print(f"\n... y {len(matches_df.columns)-10} columnas más con estadísticas avanzadas")
    
    # Verificar estadísticas avanzadas disponibles
    advanced_stats = [col for col in matches_df.columns if any(keyword in col.lower() for keyword in 
                     ['xg', 'shot', 'pass', 'possession', 'tackle', 'foul', 'corner', 'offside'])]
    
    print(f"\nESTADÍSTICAS AVANZADAS DETECTADAS ({len(advanced_stats)}):")
    for i, stat in enumerate(advanced_stats[:15]):  # Mostrar primeras 15
        print(f"   • {stat}")
    if len(advanced_stats) > 15:
        print(f"   • ... y {len(advanced_stats)-15} estadísticas más")
    
    # Mostrar distribución por temporada
    print(f"\nDISTRIBUCIÓN POR TEMPORADA:")
    season_stats = matches_df.groupby('season_id').agg({
        'match_id': 'nunique',
        'team_name': 'nunique'
    }).rename(columns={'match_id': 'partidos', 'team_name': 'equipos'})
    
    for season, stats in season_stats.iterrows():
        print(f"   {season}: {stats['partidos']} partidos, {stats['equipos']} equipos")
    
    # Verificar calidad de datos
    print(f"\nCALIDAD DE DATOS:")
    missing_percentage = (matches_df.isnull().sum().sum() / (len(matches_df) * len(matches_df.columns))) * 100
    print(f"   Datos faltantes: {missing_percentage:.2f}%")
    
    # Columnas numéricas para ML
    numeric_cols = matches_df.select_dtypes(include=[np.number]).columns
    print(f"   Columnas numéricas: {len(numeric_cols)} (para ML)")
    
    raw_data = matches_df.copy()  # Para compatibilidad con código posterior
    
except Exception as e:
    print(f" ERROR cargando datos: {e}")
    print("Intentando cargar con pipeline alternativo...")
    
    # Fallback al pipeline original
    from etl_pipeline_csv import FootballETLPipelineCSV
    data_dir = r"c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_Mining\eda_outputsMatchesPremierLeague"
    pipeline = FootballETLPipelineCSV(data_dir)
    raw_data = pipeline.extract_raw_data()
    
    if raw_data is not None:
        matches_df = raw_data.copy()
        print(" Datos cargados con pipeline alternativo")
    else:
        print(" Error en ambos métodos de carga")

print(f"\n" + "=" * 60)
print("DATOS LISTOS PARA FEATURE ENGINEERING Y MACHINE LEARNING")
print("=" * 60)

CARGA MEJORADA DE DATOS CON ESTADÍSTICAS AVANZADAS
Cargando datos desde match_data_cleaned.csv...
Este archivo contiene estadísticas más avanzadas y completas
 CARGA EXITOSA!
   Registros totales: 6,070
   Rango temporal: 2017-08-11 00:00:00 a 2025-05-25 00:00:00
   Partidos únicos: 3,035
   Equipos únicos: 31
   Temporadas: 8
   Total columnas: 93

PRIMERAS 10 COLUMNAS:
    1. match_id
    2. season_id
    3. team_id
    4. team_name
    5. home_away
    6. ttl_gls
    7. ttl_ast
    8. ttl_xg
    9. ttl_xag
   10. ttl_pk_made

... y 83 columnas más con estadísticas avanzadas

ESTADÍSTICAS AVANZADAS DETECTADAS (26):
   • ttl_xg
   • ttl_gls_xg_diff
   • ttl_pass_cmp
   • ttl_pass_att
   • pct_pass_cmp
   • ttl_pass_prog
   • ttl_key_passes
   • ttl_pass_opp_box
   • ttl_pass_live
   • ttl_pass_dead
   • ttl_pass_fk
   • ttl_pass_offside
   • ttl_pass_blocked
   • ttl_pass_rcvd
   • ttl_pass_prog_rcvd
   • ... y 11 estadísticas más

DISTRIBUCIÓN POR TEMPORADA:
   2017-2018: 378 partido

# 3.  Análisis Exploratorio de Features

Antes de entrenar los modelos, exploremos las características de nuestros features engineered.

In [37]:
# ========================================================================================
# SECCIÓN 3: FEATURE ENGINEERING OPTIMIZADO CON ESTADÍSTICAS AVANZADAS
# ========================================================================================

print("FEATURE ENGINEERING MEJORADO CON ESTADÍSTICAS AVANZADAS")
print("=" * 60)

# ========================================================================================
# CREAR DATASET DE PARTIDOS PARA MACHINE LEARNING
# ========================================================================================

def create_enhanced_match_features(df):
    """
    Crear features avanzadas usando estadísticas del CSV + features calculadas
    """
    print("Creando features avanzadas para Machine Learning...")
    
    # Ordenar por fecha para cálculos temporales
    df_sorted = df.sort_values(['date_game', 'match_id']).reset_index(drop=True)
    
    # Agrupar por match_id para crear dataset de partidos
    match_groups = df_sorted.groupby('match_id')
    
    matches_list = []
    
    for match_id, group in match_groups:
        if len(group) != 2:  # Debe tener exactamente 2 equipos
            continue
            
        # Separar home y away
        if 'is_home' in group.columns:
            home_team = group[group['is_home'] == True].iloc[0] if any(group['is_home']) else group.iloc[0]
            away_team = group[group['is_home'] == False].iloc[0] if any(~group['is_home']) else group.iloc[1]
        else:
            # Asumir que el primer registro es home
            home_team = group.iloc[0]
            away_team = group.iloc[1]
        
        # Features básicas
        match_features = {
            'match_id': match_id,
            'date_game': home_team['date_game'],
            'season_id': home_team['season_id'],
            'matchday': home_team.get('matchday', 1),
            
            # Equipos
            'home_team': home_team['team_name'],
            'away_team': away_team['team_name'],
            'home_team_id': home_team.get('team_id', home_team['team_name']),
            'away_team_id': away_team.get('team_id', away_team['team_name']),
            
            # Resultado real
            'home_goals': home_team.get('goals_for', home_team.get('gls', 0)),
            'away_goals': away_team.get('goals_for', away_team.get('gls', 0)),
        }
        
        # Determinar resultado
        if match_features['home_goals'] > match_features['away_goals']:
            match_features['result'] = 'H'
        elif match_features['home_goals'] < match_features['away_goals']:
            match_features['result'] = 'A'
        else:
            match_features['result'] = 'D'
        
        # ========================================================================================
        # FEATURES AVANZADAS DEL CSV
        # ========================================================================================
        
        # Expected Goals (xG)
        match_features['home_xg'] = home_team.get('xg', 0)
        match_features['away_xg'] = away_team.get('xg', 0)
        match_features['xg_difference'] = match_features['home_xg'] - match_features['away_xg']
        
        # Shooting Statistics
        match_features['home_shots'] = home_team.get('sh', 0)
        match_features['away_shots'] = away_team.get('sh', 0)
        match_features['home_shots_on_target'] = home_team.get('sot', 0)
        match_features['away_shots_on_target'] = away_team.get('sot', 0)
        match_features['shots_difference'] = match_features['home_shots'] - match_features['away_shots']
        
        # Possession (si está disponible)
        match_features['home_possession'] = home_team.get('possession', 50)
        match_features['away_possession'] = away_team.get('possession', 50)
        match_features['possession_difference'] = match_features['home_possession'] - match_features['away_possession']
        
        # Passing Statistics
        match_features['home_passes'] = home_team.get('pass_cmp', 0)
        match_features['away_passes'] = away_team.get('pass_cmp', 0)
        match_features['home_pass_accuracy'] = home_team.get('pass_pct', 0)
        match_features['away_pass_accuracy'] = away_team.get('pass_pct', 0)
        
        # Defensive Statistics
        match_features['home_tackles'] = home_team.get('tkl', 0)
        match_features['away_tackles'] = away_team.get('tkl', 0)
        match_features['home_interceptions'] = home_team.get('int', 0)
        match_features['away_interceptions'] = away_team.get('int', 0)
        
        # Disciplinary
        match_features['home_yellow_cards'] = home_team.get('crdy', 0)
        match_features['away_yellow_cards'] = away_team.get('crdy', 0)
        match_features['home_red_cards'] = home_team.get('crdr', 0)
        match_features['away_red_cards'] = away_team.get('crdr', 0)
        
        # Set Pieces
        match_features['home_corners'] = home_team.get('corners', 0)
        match_features['away_corners'] = away_team.get('corners', 0)
        match_features['home_fouls'] = home_team.get('fouls', 0)
        match_features['away_fouls'] = away_team.get('fouls', 0)
        
        # Ratios and advanced metrics
        match_features['home_shot_conversion'] = (match_features['home_goals'] / match_features['home_shots'] 
                                                 if match_features['home_shots'] > 0 else 0)
        match_features['away_shot_conversion'] = (match_features['away_goals'] / match_features['away_shots'] 
                                                 if match_features['away_shots'] > 0 else 0)
        
        match_features['home_xg_overperformance'] = match_features['home_goals'] - match_features['home_xg']
        match_features['away_xg_overperformance'] = match_features['away_goals'] - match_features['away_xg']
        
        # Agregar todas las columnas numéricas adicionales del CSV
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        for col in numeric_columns:
            if col not in ['match_id', 'team_id', 'season_id', 'matchday']:
                home_val = home_team.get(col, 0)
                away_val = away_team.get(col, 0)
                
                match_features[f'home_{col}'] = home_val
                match_features[f'away_{col}'] = away_val
                
                # Diferencia entre equipos
                if isinstance(home_val, (int, float)) and isinstance(away_val, (int, float)):
                    match_features[f'{col}_difference'] = home_val - away_val
        
        matches_list.append(match_features)
    
    return pd.DataFrame(matches_list)

# Crear dataset de partidos con features avanzadas
print("Procesando estadísticas avanzadas del CSV...")

matches_final = create_enhanced_match_features(matches_df)

print(f"✅ DATASET DE PARTIDOS CREADO:")
print(f"   Partidos procesados: {len(matches_final):,}")
print(f"   Features totales: {len(matches_final.columns):,}")
print(f"   Rango temporal: {matches_final['date_game'].min()} a {matches_final['date_game'].max()}")

# Mostrar distribución de resultados
result_dist = matches_final['result'].value_counts()
print(f"\nDISTRIBUCIÓN DE RESULTADOS:")
for result, count in result_dist.items():
    pct = (count / len(matches_final)) * 100
    result_name = {'H': 'Victoria Local', 'D': 'Empate', 'A': 'Victoria Visitante'}[result]
    print(f"   {result_name}: {count:,} ({pct:.1f}%)")

# Identificar features para ML
exclude_cols = ['match_id', 'date_game', 'season_id', 'home_team', 'away_team', 'home_team_id', 'away_team_id', 'result', 'home_goals', 'away_goals']
feature_cols = [col for col in matches_final.columns if col not in exclude_cols]

print(f"\nFEATURES PARA MACHINE LEARNING: {len(feature_cols)}")

# Categorizar features por tipo
feature_groups = {
    'Expected Goals': [col for col in feature_cols if 'xg' in col.lower()],
    'Shooting': [col for col in feature_cols if any(keyword in col.lower() for keyword in ['shot', 'sot'])],
    'Possession': [col for col in feature_cols if 'possession' in col.lower()],
    'Passing': [col for col in feature_cols if 'pass' in col.lower()],
    'Defensive': [col for col in feature_cols if any(keyword in col.lower() for keyword in ['tackle', 'tkl', 'int'])],
    'Disciplinary': [col for col in feature_cols if any(keyword in col.lower() for keyword in ['card', 'crdy', 'crdr'])],
    'Set Pieces': [col for col in feature_cols if any(keyword in col.lower() for keyword in ['corner', 'foul'])],
    'Differences': [col for col in feature_cols if 'difference' in col.lower()],
    'Advanced': [col for col in feature_cols if any(keyword in col.lower() for keyword in ['conversion', 'overperformance'])]
}

print(f"\nCATEGORÍAS DE FEATURES:")
for group, features in feature_groups.items():
    if features:
        print(f"   {group}: {len(features)} features")

# Verificar calidad de features
print(f"\nCALIDAD DE FEATURES:")
feature_data = matches_final[feature_cols]
missing_pct = (feature_data.isnull().sum().sum() / (len(feature_data) * len(feature_data.columns))) * 100
print(f"   Datos faltantes en features: {missing_pct:.2f}%")
print(f"   Features numéricas: {len(feature_data.select_dtypes(include=[np.number]).columns)}")

print(f"\n" + "=" * 60)
print("FEATURES AVANZADAS LISTAS PARA MODELOS DE MACHINE LEARNING")
print("=" * 60)

FEATURE ENGINEERING MEJORADO CON ESTADÍSTICAS AVANZADAS
Procesando estadísticas avanzadas del CSV...
Creando features avanzadas para Machine Learning...
✅ DATASET DE PARTIDOS CREADO:
   Partidos procesados: 3,035
   Features totales: 221
   Rango temporal: 2017-08-11 00:00:00 a 2025-05-25 00:00:00

DISTRIBUCIÓN DE RESULTADOS:
   Victoria Visitante: 1,172 (38.6%)
   Victoria Local: 1,161 (38.3%)
   Empate: 702 (23.1%)

FEATURES PARA MACHINE LEARNING: 211

CATEGORÍAS DE FEATURES:
   Expected Goals: 14 features
   Shooting: 22 features
   Possession: 9 features
   Passing: 40 features
   Defensive: 20 features
   Disciplinary: 10 features
   Set Pieces: 6 features
   Differences: 22 features
   Advanced: 7 features

CALIDAD DE FEATURES:
   Datos faltantes en features: 1.42%
   Features numéricas: 211

FEATURES AVANZADAS LISTAS PARA MODELOS DE MACHINE LEARNING
✅ DATASET DE PARTIDOS CREADO:
   Partidos procesados: 3,035
   Features totales: 221
   Rango temporal: 2017-08-11 00:00:00 a 2025-

# 3.1 Análisis de Features Avanzadas

Exploramos las estadísticas avanzadas disponibles en match_data_cleaned.csv para maximizar el rendimiento del modelo.

In [38]:
# ========================================================================================
# ANÁLISIS DETALLADO DE FEATURES AVANZADAS DISPONIBLES
# ========================================================================================

print("ANÁLISIS AVANZADO DE ESTADÍSTICAS DISPONIBLES")
print("=" * 55)

# ========================================================================================
# EXPLORAR TODAS LAS COLUMNAS DEL CSV ORIGINAL
# ========================================================================================

print("COLUMNAS DISPONIBLES EN match_data_cleaned.csv:")
all_columns = list(matches_df.columns)
print(f"Total: {len(all_columns)} columnas")

# Categorizar columnas por tipo de estadística
column_categories = {
    'Básicas': [],
    'Expected Goals (xG)': [],
    'Tiros y Remates': [],
    'Posesión': [],
    'Pases': [],
    'Defensa': [],
    'Disciplina': [],
    'Set Pieces': [],
    'Portero': [],
    'Avanzadas': []
}

for col in all_columns:
    col_lower = col.lower()
    
    if any(keyword in col_lower for keyword in ['match_id', 'team', 'date', 'season']):
        column_categories['Básicas'].append(col)
    elif 'xg' in col_lower or 'expected' in col_lower:
        column_categories['Expected Goals (xG)'].append(col)
    elif any(keyword in col_lower for keyword in ['shot', 'sot', 'sh_', 'shoot']):
        column_categories['Tiros y Remates'].append(col)
    elif any(keyword in col_lower for keyword in ['poss', 'touch']):
        column_categories['Posesión'].append(col)
    elif any(keyword in col_lower for keyword in ['pass', 'cmp', 'att', 'key']):
        column_categories['Pases'].append(col)
    elif any(keyword in col_lower for keyword in ['tkl', 'int', 'blocks', 'clear']):
        column_categories['Defensa'].append(col)
    elif any(keyword in col_lower for keyword in ['card', 'crdy', 'crdr', 'foul']):
        column_categories['Disciplina'].append(col)
    elif any(keyword in col_lower for keyword in ['corner', 'cross', 'fk']):
        column_categories['Set Pieces'].append(col)
    elif any(keyword in col_lower for keyword in ['gk', 'save', 'ga', 'keeper']):
        column_categories['Portero'].append(col)
    else:
        column_categories['Avanzadas'].append(col)

# Mostrar categorías
for category, cols in column_categories.items():
    if cols:
        print(f"\n{category} ({len(cols)} columnas):")
        for col in cols[:8]:  # Mostrar primeras 8
            sample_val = matches_df[col].iloc[0] if not matches_df[col].isnull().all() else 'N/A'
            print(f"   • {col:<25} (ej: {sample_val})")
        if len(cols) > 8:
            print(f"   • ... y {len(cols)-8} más")

# ========================================================================================
# ANÁLISIS DE CALIDAD Y COMPLETITUD
# ========================================================================================

print(f"\nANÁLISIS DE CALIDAD DE DATOS:")
print("=" * 30)

# Completitud por columna
numeric_cols = matches_df.select_dtypes(include=[np.number]).columns
completeness = {}

for col in numeric_cols:
    non_null_count = matches_df[col].notna().sum()
    total_count = len(matches_df)
    completeness[col] = (non_null_count / total_count) * 100

# Ordenar por completitud
completeness_sorted = dict(sorted(completeness.items(), key=lambda x: x[1], reverse=True))

print("COMPLETITUD DE DATOS NUMÉRICOS (Top 15):")
for i, (col, pct) in enumerate(list(completeness_sorted.items())[:15]):
    status = "✅" if pct > 95 else "⚠️" if pct > 80 else "❌"
    print(f"   {i+1:2d}. {col:<25} {pct:6.1f}% {status}")

# Columnas con datos faltantes significativos
low_quality = [(col, pct) for col, pct in completeness_sorted.items() if pct < 80]
if low_quality:
    print(f"\nCOLUMNAS CON DATOS FALTANTES (>{20}%):")
    for col, pct in low_quality[:10]:
        print(f"   • {col:<25} {pct:5.1f}%")

# ========================================================================================
# ESTADÍSTICAS DESCRIPTIVAS DE FEATURES CLAVE
# ========================================================================================

print(f"\nESTADÍSTICAS DE FEATURES CLAVE:")
print("=" * 32)

key_stats = ['gls', 'xg', 'sh', 'sot', 'pass_cmp', 'pass_att', 'tkl', 'int']
available_key_stats = [col for col in key_stats if col in matches_df.columns]

if available_key_stats:
    stats_summary = matches_df[available_key_stats].describe()
    
    for col in available_key_stats:
        print(f"\n{col.upper()}:")
        print(f"   Promedio: {stats_summary.loc['mean', col]:.2f}")
        print(f"   Mediana:  {stats_summary.loc['50%', col]:.2f}")
        print(f"   Rango:    {stats_summary.loc['min', col]:.1f} - {stats_summary.loc['max', col]:.1f}")

# ========================================================================================
# CORRELACIONES IMPORTANTES
# ========================================================================================

print(f"\nCORRELACIONES MÁS RELEVANTES:")
print("=" * 32)

# Identificar columnas para análisis de correlación
correlation_cols = [col for col in matches_df.select_dtypes(include=[np.number]).columns 
                   if col not in ['match_id', 'team_id', 'season_id']][:20]  # Limitar a 20 para rendimiento

if len(correlation_cols) > 1:
    corr_matrix = matches_df[correlation_cols].corr()
    
    # Encontrar correlaciones altas (excluyendo autocorrelaciones)
    high_correlations = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > 0.7:  # Correlación alta
                high_correlations.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_val))
    
    if high_correlations:
        print("Correlaciones > 0.7:")
        for col1, col2, corr_val in sorted(high_correlations, key=lambda x: abs(x[2]), reverse=True)[:10]:
            print(f"   {col1} ↔ {col2}: {corr_val:.3f}")
    else:
        print("No se encontraron correlaciones > 0.7")

print(f"\n" + "=" * 55)
print("ANÁLISIS DE FEATURES COMPLETADO - DATOS LISTOS PARA ML")
print("=" * 55)

ANÁLISIS AVANZADO DE ESTADÍSTICAS DISPONIBLES
COLUMNAS DISPONIBLES EN match_data_cleaned.csv:
Total: 93 columnas

Básicas (6 columnas):
   • match_id                  (ej: 00836893)
   • season_id                 (ej: 2017-2018)
   • team_id                   (ej: 4ba7cbea)
   • team_name                 (ej: Bournemouth)
   • date_game                 (ej: 2017-09-15 00:00:00)
   • season_phase              (ej: Early)

Expected Goals (xG) (3 columnas):
   • ttl_xg                    (ej: 1.2)
   • ttl_gls_xg_diff           (ej: 0.8)
   • xg_performance            (ej: 1.6666666666666667)

Tiros y Remates (7 columnas):
   • ttl_sot_ag                (ej: 1)
   • ttl_sot                   (ej: 3)
   • pct_sot                   (ej: 20.0)
   • ttl_gls_per_sot           (ej: 0.67)
   • ttl_sh_blocked            (ej: 2)
   • shot_accuracy             (ej: 20.0)
   • total_shots               (ej: 15)

Posesión (5 columnas):
   • avg_poss                  (ej: 0.0)
   • ttl_touches        

# 4.  Modelo Baseline - Sistema Elo Rating

Implementamos el primer modelo baseline utilizando el sistema Elo Rating, que es ampliamente usado en deportes para ranking de equipos.

In [39]:
# ========================================================================================
# SECCIÓN 4: SISTEMA ELO RATING BASELINE MODEL
# ========================================================================================

print("SISTEMA ELO RATING - MODELO BASELINE")
print("=" * 50)

# Implementar sistema Elo simplificado
class SimpleEloSystem:
    def __init__(self, k_factor=20, initial_rating=1500):
        self.k_factor = k_factor
        self.initial_rating = initial_rating
        self.ratings = {}
    
    def get_rating(self, team):
        if team not in self.ratings:
            self.ratings[team] = self.initial_rating
        return self.ratings[team]
    
    def calculate_expected_score(self, rating_a, rating_b):
        return 1 / (1 + 10**((rating_b - rating_a) / 400))
    
    def update_ratings(self, team_home, team_away, result):
        # Obtener ratings actuales
        rating_home = self.get_rating(team_home)
        rating_away = self.get_rating(team_away)
        
        # Calcular probabilidades esperadas
        expected_home = self.calculate_expected_score(rating_home, rating_away)
        expected_away = 1 - expected_home
        
        # Resultado real (1 = victoria local, 0.5 = empate, 0 = victoria visitante)
        if result == 'H':
            actual_home = 1.0
            actual_away = 0.0
        elif result == 'D':
            actual_home = 0.5
            actual_away = 0.5
        else:  # result == 'A'
            actual_home = 0.0
            actual_away = 1.0
        
        # Actualizar ratings
        self.ratings[team_home] = rating_home + self.k_factor * (actual_home - expected_home)
        self.ratings[team_away] = rating_away + self.k_factor * (actual_away - expected_away)
        
        return expected_home

if 'matches_final' in locals() and len(matches_final) > 0:
    
    # Inicializar sistema Elo
    elo_system = SimpleEloSystem()
    
    # Ordenar partidos por fecha
    matches_elo = matches_final.sort_values('date_game').copy()
    
    # Calcular features Elo
    elo_features = []
    
    print("Calculando ratings Elo históricos...")
    
    for idx, match in matches_elo.iterrows():
        home_team = match['home_team']
        away_team = match['away_team']
        result = match['result']
        
        # Obtener ratings antes del partido
        home_elo_before = elo_system.get_rating(home_team)
        away_elo_before = elo_system.get_rating(away_team)
        
        # Calcular probabilidad esperada
        expected_home_prob = elo_system.calculate_expected_score(home_elo_before, away_elo_before)
        
        # Actualizar ratings basado en resultado
        elo_system.update_ratings(home_team, away_team, result)
        
        # Guardar features
        elo_features.append({
            'match_id': match['match_id'],
            'home_elo_before': home_elo_before,
            'away_elo_before': away_elo_before,
            'elo_diff': home_elo_before - away_elo_before,
            'elo_home_prob': expected_home_prob,
            'elo_away_prob': 1 - expected_home_prob
        })
    
    # Convertir a DataFrame y merge con datos principales
    elo_df = pd.DataFrame(elo_features)
    matches_with_elo = matches_final.merge(elo_df, on='match_id', how='left')
    
    print(f"✅ SISTEMA ELO IMPLEMENTADO:")
    print(f"   Partidos procesados: {len(matches_with_elo):,}")
    print(f"   Equipos con rating: {len(elo_system.ratings)}")
    
    # Mostrar top 10 equipos por rating final
    print(f"\nTOP 10 EQUIPOS POR RATING ELO FINAL:")
    sorted_ratings = sorted(elo_system.ratings.items(), key=lambda x: x[1], reverse=True)
    for i, (team, rating) in enumerate(sorted_ratings[:10], 1):
        print(f"   {i:2d}. {team:<20} {rating:,.0f}")
    
    # Análisis de probabilidades
    print(f"\nANÁLISIS DE PROBABILIDADES ELO:")
    print(f"   Probabilidad promedio local: {matches_with_elo['elo_home_prob'].mean():.3f}")
    print(f"   Probabilidad mínima local: {matches_with_elo['elo_home_prob'].min():.3f}")
    print(f"   Probabilidad máxima local: {matches_with_elo['elo_home_prob'].max():.3f}")
    
    # Correlación con resultados reales
    matches_with_elo['actual_result'] = matches_with_elo['result'].map({'H': 1, 'D': 0.5, 'A': 0})
    elo_correlation = np.corrcoef(matches_with_elo['elo_home_prob'], matches_with_elo['actual_result'])[0,1]
    print(f"   Correlación Elo-Resultado: {elo_correlation:.3f}")
    
    # Actualizar dataset principal
    matches_final = matches_with_elo.copy()
    
    print(f"\n" + "=" * 50)
    print("SISTEMA ELO INTEGRADO AL DATASET PRINCIPAL")
    print("=" * 50)

else:
    print("❌ No hay datos disponibles para calcular ratings Elo")

SISTEMA ELO RATING - MODELO BASELINE
Calculando ratings Elo históricos...
✅ SISTEMA ELO IMPLEMENTADO:
   Partidos procesados: 3,035
   Equipos con rating: 31

TOP 10 EQUIPOS POR RATING ELO FINAL:
    1. Manchester City      1,741
    2. Liverpool            1,730
    3. Arsenal              1,716
    4. Aston Villa          1,646
    5. Chelsea              1,633
    6. Newcastle Utd        1,620
    7. Crystal Palace       1,593
    8. Brighton             1,586
    9. Nott'ham Forest      1,567
   10. Everton              1,557

ANÁLISIS DE PROBABILIDADES ELO:
   Probabilidad promedio local: 0.509
   Probabilidad mínima local: 0.068
   Probabilidad máxima local: 0.906
   Correlación Elo-Resultado: 0.397

SISTEMA ELO INTEGRADO AL DATASET PRINCIPAL
✅ SISTEMA ELO IMPLEMENTADO:
   Partidos procesados: 3,035
   Equipos con rating: 31

TOP 10 EQUIPOS POR RATING ELO FINAL:
    1. Manchester City      1,741
    2. Liverpool            1,730
    3. Arsenal              1,716
    4. Aston Vill

# 5.  Modelo Baseline - Distribución Poisson

El segundo modelo baseline utiliza distribución Poisson para modelar goles y predecir resultados basándose en fortalezas ofensivas y defensivas de equipos.

In [40]:
# ========================================================================================
# SECCIÓN 5: FEATURES DE FORMA Y ESTADÍSTICAS DE EQUIPO
# ========================================================================================

print("CALCULANDO FEATURES DE FORMA Y ESTADÍSTICAS AVANZADAS")
print("=" * 60)

def calculate_team_form_features(df, n_games=5):
    """
    Calcular features de forma basados en los últimos n partidos
    """
    print(f"Calculando forma de equipos (últimos {n_games} partidos)...")
    
    # Ordenar por fecha
    df_sorted = df.sort_values('date_game').copy()
    
    # Listas para almacenar features
    form_features = []
    
    # Procesar cada partido
    for idx, match in df_sorted.iterrows():
        home_team = match['home_team']
        away_team = match['away_team']
        match_date = match['date_game']
        
        # Obtener partidos anteriores para cada equipo
        previous_matches = df_sorted[df_sorted['date_game'] < match_date]
        
        # Features de forma para equipo local
        home_previous = previous_matches[
            (previous_matches['home_team'] == home_team) | 
            (previous_matches['away_team'] == home_team)
        ].tail(n_games)
        
        # Features de forma para equipo visitante  
        away_previous = previous_matches[
            (previous_matches['home_team'] == away_team) | 
            (previous_matches['away_team'] == away_team)
        ].tail(n_games)
        
        # Calcular estadísticas de forma local
        home_form_stats = calculate_form_stats(home_previous, home_team)
        away_form_stats = calculate_form_stats(away_previous, away_team)
        
        # Combinar con datos del partido
        match_features = {
            'match_id': match['match_id'],
            **{f'home_form_{k}': v for k, v in home_form_stats.items()},
            **{f'away_form_{k}': v for k, v in away_form_stats.items()}
        }
        
        form_features.append(match_features)
    
    return pd.DataFrame(form_features)

def calculate_form_stats(team_matches, team_name):
    """
    Calcular estadísticas de forma para un equipo
    """
    if len(team_matches) == 0:
        return {
            'points': 0, 'goals_for': 0, 'goals_against': 0,
            'wins': 0, 'draws': 0, 'losses': 0, 'games': 0
        }
    
    points = 0
    goals_for = 0
    goals_against = 0
    wins = draws = losses = 0
    
    for _, match in team_matches.iterrows():
        # Determinar si el equipo jugó como local o visitante
        if match['home_team'] == team_name:
            # Equipo jugó como local
            team_goals = match['home_goals']
            opponent_goals = match['away_goals']
        else:
            # Equipo jugó como visitante
            team_goals = match['away_goals']
            opponent_goals = match['home_goals']
        
        goals_for += team_goals
        goals_against += opponent_goals
        
        # Calcular puntos y estadísticas
        if team_goals > opponent_goals:
            points += 3
            wins += 1
        elif team_goals == opponent_goals:
            points += 1
            draws += 1
        else:
            losses += 1
    
    return {
        'points': points,
        'goals_for': goals_for,
        'goals_against': goals_against,
        'goal_difference': goals_for - goals_against,
        'wins': wins,
        'draws': draws,
        'losses': losses,
        'games': len(team_matches),
        'points_per_game': points / len(team_matches) if len(team_matches) > 0 else 0
    }

if 'matches_final' in locals() and len(matches_final) > 0:
    
    # Calcular features de forma
    form_df = calculate_team_form_features(matches_final, n_games=5)
    
    # Merge con dataset principal
    matches_with_form = matches_final.merge(form_df, on='match_id', how='left')
    
    # Llenar valores NaN con 0 (para primeros partidos)
    form_columns = [col for col in matches_with_form.columns if 'form' in col]
    matches_with_form[form_columns] = matches_with_form[form_columns].fillna(0)
    
    # Calcular features adicionales
    matches_with_form['form_points_diff'] = (
        matches_with_form['home_form_points'] - matches_with_form['away_form_points']
    )
    matches_with_form['form_goals_diff'] = (
        matches_with_form['home_form_goals_for'] - matches_with_form['away_form_goals_for']
    )
    matches_with_form['form_defense_diff'] = (
        matches_with_form['away_form_goals_against'] - matches_with_form['home_form_goals_against']
    )
    
    print(f"✅ FEATURES DE FORMA CALCULADOS:")
    print(f"   Partidos procesados: {len(matches_with_form):,}")
    print(f"   Features de forma: {len(form_columns)}")
    
    # Análisis de forma promedio
    print(f"\nANÁLISIS DE FORMA PROMEDIO:")
    print(f"   Puntos forma local: {matches_with_form['home_form_points'].mean():.2f}")
    print(f"   Puntos forma visitante: {matches_with_form['away_form_points'].mean():.2f}")
    print(f"   Goles forma local: {matches_with_form['home_form_goals_for'].mean():.2f}")
    print(f"   Goles forma visitante: {matches_with_form['away_form_goals_for'].mean():.2f}")
    
    # Mostrar distribución de diferencias de forma
    print(f"\nDISTRIBUCIÓN DE DIFERENCIAS:")
    print(f"   Diferencia puntos: {matches_with_form['form_points_diff'].mean():.2f} ± {matches_with_form['form_points_diff'].std():.2f}")
    print(f"   Diferencia goles: {matches_with_form['form_goals_diff'].mean():.2f} ± {matches_with_form['form_goals_diff'].std():.2f}")
    
    # Actualizar dataset principal
    matches_final = matches_with_form.copy()
    
    # Contar features finales para ML
    exclude_cols = [
        'match_id', 'date_game', 'season_id', 'matchday',
        'home_team', 'away_team', 'home_team_id', 'away_team_id',
        'result', 'home_goals', 'away_goals'
    ]
    
    ml_features = [col for col in matches_final.columns if col not in exclude_cols]
    
    print(f"\nFEATURES TOTALES PARA ML: {len(ml_features)}")
    print("   Categorías de features:")
    print(f"   • Estadísticas avanzadas: {len([f for f in ml_features if not any(x in f for x in ['elo', 'form'])])}")
    print(f"   • Elo ratings: {len([f for f in ml_features if 'elo' in f])}")
    print(f"   • Forma de equipos: {len([f for f in ml_features if 'form' in f])}")
    
    print(f"\n" + "=" * 60)
    print("DATASET FINAL PREPARADO PARA MACHINE LEARNING")
    print("=" * 60)

else:
    print("❌ No hay datos disponibles para calcular features de forma")

CALCULANDO FEATURES DE FORMA Y ESTADÍSTICAS AVANZADAS
Calculando forma de equipos (últimos 5 partidos)...
✅ FEATURES DE FORMA CALCULADOS:
   Partidos procesados: 3,035
   Features de forma: 23

ANÁLISIS DE FORMA PROMEDIO:
   Puntos forma local: 6.87
   Puntos forma visitante: 6.81
   Goles forma local: 6.87
   Goles forma visitante: 6.72

DISTRIBUCIÓN DE DIFERENCIAS:
   Diferencia puntos: 0.05 ± 4.98
   Diferencia goles: 0.15 ± 4.78

FEATURES TOTALES PARA ML: 237
   Categorías de features:
   • Estadísticas avanzadas: 206
   • Elo ratings: 5
   • Forma de equipos: 26

DATASET FINAL PREPARADO PARA MACHINE LEARNING
✅ FEATURES DE FORMA CALCULADOS:
   Partidos procesados: 3,035
   Features de forma: 23

ANÁLISIS DE FORMA PROMEDIO:
   Puntos forma local: 6.87
   Puntos forma visitante: 6.81
   Goles forma local: 6.87
   Goles forma visitante: 6.72

DISTRIBUCIÓN DE DIFERENCIAS:
   Diferencia puntos: 0.05 ± 4.98
   Diferencia goles: 0.15 ± 4.78

FEATURES TOTALES PARA ML: 237
   Categorías de 

# 6.  Modelo XGBoost - Entrenamiento y Optimización

Ahora implementamos nuestro modelo principal XGBoost con todos los features engineered y optimización automática de hiperparámetros.

In [41]:
# ========================================================================================
# SECCIÓN 6: MODELOS BASELINE (ELO + POISSON)  
# ========================================================================================

print("ENTRENANDO MODELOS BASELINE")
print("=" * 40)

# Implementación simplificada de modelos baseline
class SimpleEloModel:
    def __init__(self):
        self.name = "Elo Baseline"
        
    def predict(self, X):
        """Predecir basado en probabilidades Elo"""
        predictions = []
        for _, row in X.iterrows():
            if 'elo_home_prob' in row:
                prob_home = row['elo_home_prob']
                if prob_home > 0.55:
                    predictions.append('H')
                elif prob_home < 0.40:
                    predictions.append('A')
                else:
                    predictions.append('D')
            else:
                # Fallback basado en diferencia Elo
                if 'elo_diff' in row:
                    diff = row['elo_diff']
                    if diff > 100:
                        predictions.append('H')
                    elif diff < -100:
                        predictions.append('A')
                    else:
                        predictions.append('D')
                else:
                    predictions.append('D')  # Empate por defecto
        return predictions

class SimplePoissonModel:
    def __init__(self):
        self.name = "Poisson Baseline"
        self.home_avg_goals = 1.5
        self.away_avg_goals = 1.2
        
    def fit(self, X, y):
        """Entrenar modelo calculando promedios"""
        if 'home_goals' in X.columns and 'away_goals' in X.columns:
            self.home_avg_goals = X['home_goals'].mean()
            self.away_avg_goals = X['away_goals'].mean()
        
    def predict(self, X):
        """Predecir basado en Expected Goals o promedios"""
        predictions = []
        for _, row in X.iterrows():
            # Usar xG si está disponible
            if 'home_xg' in row and 'away_xg' in row:
                home_expected = row['home_xg']
                away_expected = row['away_xg']
            else:
                # Usar promedios
                home_expected = self.home_avg_goals
                away_expected = self.away_avg_goals
            
            # Predecir basado en expected goals
            if home_expected > away_expected * 1.3:
                predictions.append('H')
            elif away_expected > home_expected * 1.2:
                predictions.append('A')
            else:
                predictions.append('D')
                
        return predictions

if 'matches_final' in locals() and len(matches_final) > 100:
    
    # División temporal de datos (80% entrenamiento, 20% prueba)
    split_date = matches_final['date_game'].quantile(0.8)
    train_df = matches_final[matches_final['date_game'] <= split_date].copy()
    test_df = matches_final[matches_final['date_game'] > split_date].copy()
    
    print(f"División temporal:")
    print(f"   Entrenamiento: {len(train_df):,} partidos (hasta {split_date.date()})")
    print(f"   Prueba: {len(test_df):,} partidos (desde {split_date.date()})")
    
    # ================================
    # MODELO ELO BASELINE
    # ================================
    print(f"\nMODELO ELO BASELINE")
    
    elo_model = SimpleEloModel()
    elo_predictions = elo_model.predict(test_df)
    
    # Calcular accuracy
    y_test = test_df['result']
    elo_accuracy = sum(1 for pred, actual in zip(elo_predictions, y_test) if pred == actual) / len(y_test)
    
    print(f"   Modelo Elo entrenado")
    print(f"   Precisión: {elo_accuracy:.4f}")
    print(f"   Predicciones: {len(elo_predictions):,}")
    
    # ================================
    # MODELO POISSON BASELINE
    # ================================
    print(f"\nMODELO POISSON BASELINE")
    
    poisson_model = SimplePoissonModel()
    poisson_model.fit(train_df, train_df['result'])
    poisson_predictions = poisson_model.predict(test_df)
    
    # Calcular accuracy
    poisson_accuracy = sum(1 for pred, actual in zip(poisson_predictions, y_test) if pred == actual) / len(y_test)
    
    print(f"   Modelo Poisson entrenado")
    print(f"   Precisión: {poisson_accuracy:.4f}")
    print(f"   Predicciones: {len(poisson_predictions):,}")
    
    # ================================
    # COMPARACIÓN BASELINE
    # ================================
    print(f"\nCOMPARACIÓN MODELOS BASELINE:")
    print(f"   Elo Baseline:     {elo_accuracy:.4f}")
    print(f"   Poisson Baseline: {poisson_accuracy:.4f}")
    
    if elo_accuracy > poisson_accuracy:
        print(f"   🏆 Ganador: Elo (+{(elo_accuracy-poisson_accuracy):.4f})")
    else:
        print(f"   🏆 Ganador: Poisson (+{(poisson_accuracy-elo_accuracy):.4f})")
    
    # Análisis de distribución de predicciones
    print(f"\nDISTRIBUCIÓN DE PREDICCIONES:")
    
    elo_dist = pd.Series(elo_predictions).value_counts()
    poisson_dist = pd.Series(poisson_predictions).value_counts()
    real_dist = y_test.value_counts()
    
    result_names = {'H': 'Local', 'D': 'Empate', 'A': 'Visitante'}
    
    print(f"   {'Resultado':<10} {'Real':<8} {'Elo':<8} {'Poisson':<8}")
    print(f"   {'-'*10} {'-'*8} {'-'*8} {'-'*8}")
    
    for result in ['H', 'D', 'A']:
        real_count = real_dist.get(result, 0)
        elo_count = elo_dist.get(result, 0)
        poisson_count = poisson_dist.get(result, 0)
        
        real_pct = (real_count / len(y_test)) * 100
        elo_pct = (elo_count / len(elo_predictions)) * 100
        poisson_pct = (poisson_count / len(poisson_predictions)) * 100
        
        print(f"   {result_names[result]:<10} {real_pct:5.1f}%   {elo_pct:5.1f}%   {poisson_pct:5.1f}%")
    
    # Mostrar ejemplos de predicciones
    print(f"\nEJEMPLOS DE PREDICCIONES:")
    examples = test_df[['home_team', 'away_team', 'result']].head(8).copy()
    examples['elo_pred'] = elo_predictions[:8]
    examples['poisson_pred'] = poisson_predictions[:8]
    
    # Marcar aciertos
    examples['elo_ok'] = examples['result'] == examples['elo_pred']
    examples['poisson_ok'] = examples['result'] == examples['poisson_pred']
    
    print(examples[['home_team', 'away_team', 'result', 'elo_pred', 'elo_ok', 'poisson_pred', 'poisson_ok']].to_string(index=False))
    
    print(f"\n" + "=" * 40)
    print("MODELOS BASELINE COMPLETADOS")
    print("=" * 40)

else:
    print("❌ Datos insuficientes para entrenar modelos baseline")

ENTRENANDO MODELOS BASELINE
División temporal:
   Entrenamiento: 2,432 partidos (hasta 2023-12-09)
   Prueba: 603 partidos (desde 2023-12-09)

MODELO ELO BASELINE
   Modelo Elo entrenado
   Precisión: 0.4909
   Predicciones: 603

MODELO POISSON BASELINE
   Modelo Poisson entrenado
   Precisión: 0.2438
   Predicciones: 603

COMPARACIÓN MODELOS BASELINE:
   Elo Baseline:     0.4909
   Poisson Baseline: 0.2438
   🏆 Ganador: Elo (+0.2471)

DISTRIBUCIÓN DE PREDICCIONES:
   Resultado  Real     Elo      Poisson 
   ---------- -------- -------- --------
   Local       37.5%    44.9%     0.0%
   Empate      24.4%    28.9%   100.0%
   Visitante   38.1%    26.2%     0.0%

EJEMPLOS DE PREDICCIONES:
      home_team      away_team result elo_pred  elo_ok poisson_pred  poisson_ok
        Arsenal Crystal Palace      H        H    True            D       False
        Arsenal    Bournemouth      H        H    True            D       False
Manchester City        Everton      H        H    True          

# 7. Feature Importance - Análisis XGBoost

Analicemos qué features son más importantes para nuestro modelo XGBoost.

In [42]:
# ========================================================================================
# SECCIÓN 7: MODELO XGBOOST AVANZADO
# ========================================================================================

print("ENTRENANDO MODELO XGBOOST AVANZADO")
print("=" * 45)

# Importar XGBoost y scikit-learn
try:
    import xgboost as xgb
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    print("Librerías XGBoost importadas correctamente")
except ImportError as e:
    print(f"Error importando librerías: {e}")
    print("Instalando XGBoost...")

if 'train_df' in locals() and 'test_df' in locals() and len(train_df) > 100:
    
    try:
        # ================================
        # PREPARAR DATOS PARA XGBOOST
        # ================================
        print("\nPreparando datos para XGBoost...")
        
        # Definir columnas a excluir
        exclude_cols = [
            'match_id', 'date_game', 'season_id', 'matchday',
            'home_team', 'away_team', 'home_team_id', 'away_team_id',
            'result', 'home_goals', 'away_goals', 'actual_result'
        ]
        
        # Seleccionar features numéricas válidas
        all_features = [col for col in train_df.columns if col not in exclude_cols]
        
        # Filtrar solo columnas numéricas y sin NaN
        numeric_features = []
        for col in all_features:
            if train_df[col].dtype in ['int64', 'float64', 'int32', 'float32']:
                if not train_df[col].isna().all() and not test_df[col].isna().all():
                    numeric_features.append(col)
        
        print(f"   Features seleccionadas: {len(numeric_features)}")
        
        # Preparar matrices de features
        X_train = train_df[numeric_features].copy()
        X_test = test_df[numeric_features].copy()
        
        # Manejar valores NaN
        X_train = X_train.fillna(X_train.median())
        X_test = X_test.fillna(X_train.median())  # Usar mediana del train
        
        # Targets
        y_train = train_df['result'].copy()
        y_test = test_df['result'].copy()
        
        # Codificar targets (H=0, D=1, A=2)
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)
        
        print(f"   Entrenamiento: {X_train.shape[0]:,} partidos, {X_train.shape[1]} features")
        print(f"   Prueba: {X_test.shape[0]:,} partidos")
        print(f"   Clases: {list(label_encoder.classes_)}")
        
        # ================================
        # ENTRENAR MODELO XGBOOST
        # ================================
        print(f"\nEntrenando XGBoost...")
        
        # Parámetros optimizados para clasificación multiclase
        xgb_params = {
            'objective': 'multi:softprob',
            'num_class': 3,
            'max_depth': 6,
            'learning_rate': 0.1,
            'n_estimators': 100,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'eval_metric': 'mlogloss'
        }
        
        # Crear y entrenar modelo
        xgb_model = xgb.XGBClassifier(**xgb_params)
        xgb_model.fit(
            X_train, y_train_encoded,
            eval_set=[(X_test, y_test_encoded)],
            verbose=False
        )
        
        # ================================
        # PREDICCIONES Y EVALUACIÓN
        # ================================
        print("Generando predicciones...")
        
        # Predicciones
        y_pred_encoded = xgb_model.predict(X_test)
        y_pred = label_encoder.inverse_transform(y_pred_encoded)
        
        # Probabilidades
        y_pred_proba = xgb_model.predict_proba(X_test)
        
        # Accuracy
        xgb_accuracy = accuracy_score(y_test, y_pred)
        
        print(f"✅ MODELO XGBOOST ENTRENADO:")
        print(f"   Precisión: {xgb_accuracy:.4f}")
        print(f"   Predicciones: {len(y_pred):,}")
        
        # ================================
        # FEATURE IMPORTANCE
        # ================================
        print(f"\nTOP 15 FEATURES MÁS IMPORTANTES:")
        
        feature_importance = xgb_model.feature_importances_
        feature_names = numeric_features
        
        # Crear lista ordenada de importancia
        importance_pairs = list(zip(feature_names, feature_importance))
        importance_pairs.sort(key=lambda x: x[1], reverse=True)
        
        for i, (feature, importance) in enumerate(importance_pairs[:15], 1):
            print(f"   {i:2d}. {feature:<30} {importance:.4f}")
        
        # ================================
        # COMPARACIÓN FINAL DE MODELOS
        # ================================
        print(f"\nCOMPARACIÓN FINAL DE MODELOS:")
        print(f"   Elo Baseline:     {elo_accuracy:.4f}")
        print(f"   Poisson Baseline: {poisson_accuracy:.4f}")
        print(f"   XGBoost Avanzado: {xgb_accuracy:.4f}")
        
        # Determinar ganador
        best_accuracy = max(elo_accuracy, poisson_accuracy, xgb_accuracy)
        
        if best_accuracy == xgb_accuracy:
            improvement_elo = xgb_accuracy - elo_accuracy
            improvement_poisson = xgb_accuracy - poisson_accuracy
            print(f"   🏆 GANADOR: XGBoost")
            print(f"      Mejora vs Elo: +{improvement_elo:.4f}")
            print(f"      Mejora vs Poisson: +{improvement_poisson:.4f}")
        elif best_accuracy == elo_accuracy:
            print(f"   🏆 GANADOR: Elo Baseline")
        else:
            print(f"   🏆 GANADOR: Poisson Baseline")
        
        # ================================
        # ANÁLISIS DETALLADO XGBOOST
        # ================================
        print(f"\nANÁLISIS DETALLADO XGBOOST:")
        
        # Distribución de predicciones
        xgb_dist = pd.Series(y_pred).value_counts()
        real_dist = y_test.value_counts()
        
        print(f"   Distribución de predicciones:")
        result_names = {'H': 'Local', 'D': 'Empate', 'A': 'Visitante'}
        
        for result in ['H', 'D', 'A']:
            real_count = real_dist.get(result, 0)
            pred_count = xgb_dist.get(result, 0)
            real_pct = (real_count / len(y_test)) * 100
            pred_pct = (pred_count / len(y_pred)) * 100
            
            print(f"      {result_names[result]}: Real {real_pct:.1f}% | Pred {pred_pct:.1f}%")
        
        # Matriz de confusión
        cm = confusion_matrix(y_test, y_pred, labels=['H', 'D', 'A'])
        print(f"\n   Matriz de confusión:")
        print(f"      Pred\\Real  H     D     A")
        for i, pred_label in enumerate(['H', 'D', 'A']):
            values = "  ".join(f"{cm[i,j]:4d}" for j in range(3))
            print(f"      {pred_label}        {values}")
        
        # Ejemplos comparativos
        print(f"\nCOMPARACIÓN DE PREDICCIONES (8 ejemplos):")
        comparison = test_df[['home_team', 'away_team', 'result']].head(8).copy()
        comparison['elo'] = elo_predictions[:8]
        comparison['poisson'] = poisson_predictions[:8]
        comparison['xgboost'] = y_pred[:8]
        
        # Marcar aciertos
        comparison['xgb_ok'] = comparison['result'] == comparison['xgboost']
        
        print(comparison[['home_team', 'away_team', 'result', 'elo', 'poisson', 'xgboost', 'xgb_ok']].to_string(index=False))
        
        print(f"\n" + "=" * 45)
        print("MODELO XGBOOST COMPLETADO CON ÉXITO")
        print("=" * 45)
        
    except Exception as e:
        print(f"❌ Error en modelo XGBoost: {e}")
        print(f"   Tipo de error: {type(e).__name__}")
        
        # Valores de fallback
        xgb_accuracy = 0
        y_pred = ['D'] * len(test_df)
        
        print(f"\nUSANDO VALORES DE FALLBACK:")
        print(f"   XGBoost Accuracy: {xgb_accuracy:.4f}")
        print(f"   Continuando con modelos baseline...")

else:
    print("❌ Datos de entrenamiento no disponibles")
    xgb_accuracy = 0

ENTRENANDO MODELO XGBOOST AVANZADO
Librerías XGBoost importadas correctamente

Preparando datos para XGBoost...
   Features seleccionadas: 233
   Entrenamiento: 2,432 partidos, 233 features
   Prueba: 603 partidos
   Clases: ['A', 'D', 'H']

Entrenando XGBoost...
Generando predicciones...
✅ MODELO XGBOOST ENTRENADO:
   Precisión: 0.9900
   Predicciones: 603

TOP 15 FEATURES MÁS IMPORTANTES:
    1. ttl_gls_per_sot_difference     0.1403
    2. goal_conversion_difference     0.1272
    3. away_goals_for                 0.0975
    4. away_goals_against             0.0621
    5. home_ttl_gls                   0.0360
    6. xg_performance_difference      0.0321
    7. away_xg_overperformance        0.0314
    8. home_xg_overperformance        0.0305
    9. away_ttl_gls                   0.0288
   10. home_xg_performance            0.0280
   11. home_ttl_gls_ag                0.0231
   12. away_xg_performance            0.0224
   13. ttl_xg_difference              0.0148
   14. away_ttl_gls_p

# 7.5 Predicción de Marcador Exacto

Implementación de predicción de goles exactos usando el modelo más efectivo, devolviendo una sola predicción optimizada.

In [46]:
# ========================================================================================
# SECCIÓN 7.5: PREDICCIÓN DE MARCADOR EXACTO - MODELO OPTIMIZADO
# ========================================================================================

print("ENTRENANDO MODELO DE PREDICCIÓN DE MARCADOR EXACTO")
print("=" * 55)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

if 'train_df' in locals() and 'test_df' in locals() and 'numeric_features' in locals():
    
    try:
        # ================================
        # PREPARAR DATOS PARA PREDICCIÓN DE GOLES
        # ================================
        print("Preparando datos para predicción de marcador exacto...")
        
        # Usar las mismas features que el modelo XGBoost exitoso
        X_train_goals = X_train.copy()
        X_test_goals = X_test.copy()
        
        # Targets: goles separados
        y_train_home_goals = train_df['home_goals'].copy()
        y_train_away_goals = train_df['away_goals'].copy()
        y_test_home_goals = test_df['home_goals'].copy()
        y_test_away_goals = test_df['away_goals'].copy()
        
        print(f"   Features utilizadas: {X_train_goals.shape[1]}")
        print(f"   Partidos entrenamiento: {len(X_train_goals):,}")
        print(f"   Partidos prueba: {len(X_test_goals):,}")
        
        # ================================
        # ENTRENAR MODELOS DE REGRESIÓN OPTIMIZADOS
        # ================================
        print("Entrenando modelos de regresión para goles...")
        
        # Parámetros optimizados basados en experiencia con datos de fútbol
        rf_params = {
            'n_estimators': 150,
            'max_depth': 12,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'random_state': 42,
            'n_jobs': -1
        }
        
        # Modelo para goles locales
        home_goals_model = RandomForestRegressor(**rf_params)
        home_goals_model.fit(X_train_goals, y_train_home_goals)
        
        # Modelo para goles visitantes
        away_goals_model = RandomForestRegressor(**rf_params)
        away_goals_model.fit(X_train_goals, y_train_away_goals)
        
        print("   Modelos de goles entrenados exitosamente")
        
        # ================================
        # GENERAR PREDICCIONES OPTIMIZADAS
        # ================================
        print("Generando predicciones de marcador...")
        
        # Predicciones base
        pred_home_raw = home_goals_model.predict(X_test_goals)
        pred_away_raw = away_goals_model.predict(X_test_goals)
        
        # Aplicar lógica inteligente de redondeo
        pred_home_goals = []
        pred_away_goals = []
        
        for h_raw, a_raw in zip(pred_home_raw, pred_away_raw):
            # Redondear con lógica de fútbol
            home_rounded = max(0, round(h_raw))
            away_rounded = max(0, round(a_raw))
            
            # Ajustar casos extremos (muy pocos partidos >5 goles)
            home_final = min(home_rounded, 5)
            away_final = min(away_rounded, 5)
            
            pred_home_goals.append(home_final)
            pred_away_goals.append(away_final)
        
        pred_home_goals = np.array(pred_home_goals)
        pred_away_goals = np.array(pred_away_goals)
        
        # ================================
        # EVALUAR PRECISIÓN DEL MODELO
        # ================================
        print("Evaluando precisión de predicción de goles...")
        
        # Métricas de precisión
        mae_home = mean_absolute_error(y_test_home_goals, pred_home_goals)
        mae_away = mean_absolute_error(y_test_away_goals, pred_away_goals)
        rmse_home = np.sqrt(mean_squared_error(y_test_home_goals, pred_home_goals))
        rmse_away = np.sqrt(mean_squared_error(y_test_away_goals, pred_away_goals))
        
        print(f"   Goles Locales  - MAE: {mae_home:.3f} | RMSE: {rmse_home:.3f}")
        print(f"   Goles Visitante - MAE: {mae_away:.3f} | RMSE: {rmse_away:.3f}")
        
        # Accuracy de marcador exacto
        exact_matches = sum(1 for i in range(len(pred_home_goals)) 
                           if pred_home_goals[i] == y_test_home_goals.iloc[i] and 
                              pred_away_goals[i] == y_test_away_goals.iloc[i])
        exact_accuracy = exact_matches / len(pred_home_goals)
        
        print(f"   Marcador exacto: {exact_accuracy:.4f} ({exact_matches}/{len(pred_home_goals)})")
        
        # ================================
        # GENERAR RESULTADO BASADO EN MARCADOR
        # ================================
        
        # Convertir goles a resultado
        score_based_results = []
        for home_g, away_g in zip(pred_home_goals, pred_away_goals):
            if home_g > away_g:
                score_based_results.append('H')
            elif home_g < away_g:
                score_based_results.append('A')
            else:
                score_based_results.append('D')
        
        # Accuracy del resultado basado en marcador
        score_result_accuracy = accuracy_score(y_test, score_based_results)
        
        print(f"   Resultado desde marcador: {score_result_accuracy:.4f}")
        
        # ================================
        # FUNCIÓN DE PREDICCIÓN FINAL OPTIMIZADA
        # ================================
        
        def predict_match_complete(home_team, away_team):
            """
            Función final que devuelve la mejor predicción combinada
            """
            try:
                # Obtener últimos datos de los equipos
                home_recent = matches_final[matches_final['home_team'] == home_team].tail(1)
                away_recent = matches_final[matches_final['away_team'] == away_team].tail(1)
                
                if len(home_recent) == 0 or len(away_recent) == 0:
                    # Usar promedio de liga si no hay datos
                    avg_home_goals = train_df['home_goals'].mean()
                    avg_away_goals = train_df['away_goals'].mean()
                    pred_home = round(avg_home_goals)
                    pred_away = round(avg_away_goals)
                else:
                    # Usar modelo entrenado con datos reales
                    # Tomar características promedio para simular el partido
                    sample_features = X_test_goals.iloc[0:1].copy()
                    
                    # Predecir con modelos entrenados
                    pred_home_raw = home_goals_model.predict(sample_features)[0]
                    pred_away_raw = away_goals_model.predict(sample_features)[0]
                    
                    pred_home = max(0, min(5, round(pred_home_raw)))
                    pred_away = max(0, min(5, round(pred_away_raw)))
                
                # Determinar resultado
                if pred_home > pred_away:
                    result = 'Victoria Local'
                    result_code = 'H'
                elif pred_home < pred_away:
                    result = 'Victoria Visitante'
                    result_code = 'A'
                else:
                    result = 'Empate'
                    result_code = 'D'
                
                # Obtener rating Elo para confianza
                home_elo = elo_system.get_rating(home_team)
                away_elo = elo_system.get_rating(away_team)
                elo_diff = abs(home_elo - away_elo)
                
                # Calcular confianza basada en diferencia Elo
                if elo_diff > 200:
                    confidence = "Alta"
                elif elo_diff > 100:
                    confidence = "Media"
                else:
                    confidence = "Baja"
                
                return {
                    'home_team': home_team,
                    'away_team': away_team,
                    'predicted_score': f"{pred_home}-{pred_away}",
                    'home_goals': pred_home,
                    'away_goals': pred_away,
                    'result': result,
                    'result_code': result_code,
                    'confidence': confidence,
                    'home_elo': home_elo,
                    'away_elo': away_elo
                }
                
            except Exception as e:
                print(f"Error en predicción: {e}")
                return None
        
        # ================================
        # EJEMPLOS DE PREDICCIÓN FINAL
        # ================================
        print("\nEJEMPLOS DE PREDICCIÓN FINAL OPTIMIZADA:")
        print("-" * 50)
        
        # Mostrar algunas predicciones reales del conjunto de prueba
        print(f"{'Partido':<30} {'Real':<8} {'Predicho':<10} {'Resultado':<15} {'Exacto'}")
        print("-" * 75)
        
        for i in range(min(10, len(test_df))):
            match_row = test_df.iloc[i]
            home_team = match_row['home_team']
            away_team = match_row['away_team']
            real_score = f"{match_row['home_goals']}-{match_row['away_goals']}"
            pred_score = f"{pred_home_goals[i]}-{pred_away_goals[i]}"
            real_result = match_row['result']
            pred_result = score_based_results[i]
            
            # Verificar si es exacto
            is_exact = (pred_home_goals[i] == match_row['home_goals'] and 
                       pred_away_goals[i] == match_row['away_goals'])
            exact_mark = "SI" if is_exact else "NO"
            
            # Resultado en texto
            result_text = {'H': 'Local', 'D': 'Empate', 'A': 'Visitante'}[pred_result]
            
            match_name = f"{home_team[:12]} vs {away_team[:12]}"
            print(f"{match_name:<30} {real_score:<8} {pred_score:<10} {result_text:<15} {exact_mark}")
        
        # ================================
        # ESTADÍSTICAS FINALES
        # ================================
        print(f"\nESTADÍSTICAS DEL MODELO FINAL:")
        print("-" * 35)
        print(f"   Precisión marcador exacto: {exact_accuracy:.4f}")
        print(f"   Precisión resultado: {score_result_accuracy:.4f}")
        print(f"   Error promedio goles local: {mae_home:.3f}")
        print(f"   Error promedio goles visitante: {mae_away:.3f}")
        print(f"   Partidos evaluados: {len(pred_home_goals):,}")
        
        # Comparar con XGBoost resultado
        if 'xgb_accuracy' in locals():
            print(f"   XGBoost resultado directo: {xgb_accuracy:.4f}")
            
            if score_result_accuracy > xgb_accuracy:
                print("   Marcador supera a XGBoost directo")
            else:
                print("   XGBoost directo supera a marcador")
        
        # Guardar modelos para uso posterior
        score_prediction_models = {
            'home_goals_model': home_goals_model,
            'away_goals_model': away_goals_model,
            'predict_function': predict_match_complete,
            'exact_accuracy': exact_accuracy,
            'result_accuracy': score_result_accuracy
        }
        
        print(f"\n" + "=" * 55)
        print("MODELO DE PREDICCIÓN DE MARCADOR EXACTO COMPLETADO")
        print("=" * 55)
        
    except Exception as e:
        print(f"Error en implementación de marcador: {e}")
        print(f"Tipo de error: {type(e).__name__}")
        score_prediction_models = None

else:
    print("Variables necesarias no disponibles")
    score_prediction_models = None

ENTRENANDO MODELO DE PREDICCIÓN DE MARCADOR EXACTO
Preparando datos para predicción de marcador exacto...
   Features utilizadas: 233
   Partidos entrenamiento: 2,432
   Partidos prueba: 603
Entrenando modelos de regresión para goles...
   Modelos de goles entrenados exitosamente
Generando predicciones de marcador...
Evaluando precisión de predicción de goles...
   Goles Locales  - MAE: 0.003 | RMSE: 0.058
   Goles Visitante - MAE: 0.007 | RMSE: 0.081
   Marcador exacto: 0.9900 (597/603)
   Resultado desde marcador: 1.0000

EJEMPLOS DE PREDICCIÓN FINAL OPTIMIZADA:
--------------------------------------------------
Partido                        Real     Predicho   Resultado       Exacto
---------------------------------------------------------------------------
Arsenal vs Crystal Pala        5-0      5-0        Local           SI
Arsenal vs Bournemouth         3-0      3-0        Local           SI
Manchester C vs Everton        2-0      2-0        Local           SI
Burnley vs Everton

# 7.6 Demostración de Predicción Final

Sistema de predicción unificado que combina resultado y marcador en una sola respuesta optimizada.

In [48]:
# ========================================================================================
# SECCIÓN 7.6: SISTEMA DE PREDICCIÓN FINAL UNIFICADO
# ========================================================================================

print("SISTEMA DE PREDICCIÓN FINAL - UNA SOLA RESPUESTA OPTIMIZADA")
print("=" * 65)

def get_best_prediction(home_team, away_team):
    """
    Función principal que devuelve LA MEJOR predicción única
    Combina XGBoost (resultado) + Random Forest (marcador)
    """
    
    if 'score_prediction_models' not in globals() or score_prediction_models is None:
        return None
    
    try:
        # ================================
        # PREPARAR FEATURES PARA PREDICCIÓN
        # ================================
        
        # Obtener ratings Elo actuales
        home_elo = elo_system.get_rating(home_team) if 'elo_system' in globals() else 1500
        away_elo = elo_system.get_rating(away_team) if 'elo_system' in globals() else 1500
        elo_diff = home_elo - away_elo
        
        # Obtener forma reciente de equipos
        recent_matches = matches_final.tail(200) if 'matches_final' in globals() else None
        
        if recent_matches is not None:
            home_recent = recent_matches[
                (recent_matches['home_team'] == home_team) | 
                (recent_matches['away_team'] == home_team)
            ].tail(5)
            
            away_recent = recent_matches[
                (recent_matches['home_team'] == away_team) | 
                (recent_matches['away_team'] == away_team)
            ].tail(5)
        else:
            home_recent = pd.DataFrame()
            away_recent = pd.DataFrame()
        
        # Calcular promedios de rendimiento
        if len(home_recent) > 0:
            home_avg_goals = home_recent.apply(lambda row: 
                row['home_goals'] if row['home_team'] == home_team else row['away_goals'], axis=1).mean()
        else:
            home_avg_goals = 1.5  # Promedio de liga
        
        if len(away_recent) > 0:
            away_avg_goals = away_recent.apply(lambda row: 
                row['away_goals'] if row['away_team'] == away_team else row['home_goals'], axis=1).mean()
        else:
            away_avg_goals = 1.2  # Promedio de liga visitante
        
        # ================================
        # PREDICCIÓN DE MARCADOR (RANDOM FOREST)
        # ================================
        
        # Crear features simuladas basadas en promedio
        if 'X_test_goals' in globals() and len(X_test_goals) > 0:
            # Usar una muestra representativa
            sample_features = X_test_goals.iloc[0:1].copy()
            
            # Ajustar algunas features clave basadas en los equipos
            if 'elo_diff' in sample_features.columns:
                sample_features['elo_diff'] = elo_diff
            if 'elo_home_prob' in sample_features.columns:
                elo_prob = 1 / (1 + 10**(elo_diff / 400))
                sample_features['elo_home_prob'] = elo_prob
            
            # Predecir goles con modelos entrenados
            pred_home_raw = score_prediction_models['home_goals_model'].predict(sample_features)[0]
            pred_away_raw = score_prediction_models['away_goals_model'].predict(sample_features)[0]
            
            # Aplicar lógica inteligente
            pred_home = max(0, min(5, round(pred_home_raw)))
            pred_away = max(0, min(5, round(pred_away_raw)))
        else:
            # Fallback usando promedios
            pred_home = max(0, round(home_avg_goals))
            pred_away = max(0, round(away_avg_goals))
        
        # ================================
        # PREDICCIÓN DE RESULTADO (XGBOOST)
        # ================================
        
        # Determinar resultado más probable
        if pred_home > pred_away:
            primary_result = 'H'
            result_text = 'Victoria Local'
            confidence_base = 0.7
        elif pred_away > pred_home:
            primary_result = 'A'
            result_text = 'Victoria Visitante'
            confidence_base = 0.7
        else:
            primary_result = 'D'
            result_text = 'Empate'
            confidence_base = 0.6
        
        # ================================
        # CALCULAR CONFIANZA INTELIGENTE
        # ================================
        
        # Factores de confianza
        elo_confidence = min(1.0, abs(elo_diff) / 300)  # 0 a 1
        goal_confidence = abs(pred_home - pred_away) / 3.0  # Diferencia de goles
        
        # Confianza final combinada
        final_confidence = (confidence_base + elo_confidence + goal_confidence) / 3
        final_confidence = max(0.3, min(0.95, final_confidence))  # Entre 30% y 95%
        
        # Categorizar confianza
        if final_confidence >= 0.8:
            confidence_level = "Muy Alta"
        elif final_confidence >= 0.7:
            confidence_level = "Alta"
        elif final_confidence >= 0.6:
            confidence_level = "Media"
        else:
            confidence_level = "Baja"
        
        # ================================
        # RESPUESTA FINAL UNIFICADA
        # ================================
        
        prediction_result = {
            # Información básica
            'partido': f"{home_team} vs {away_team}",
            'home_team': home_team,
            'away_team': away_team,
            
            # Predicción principal
            'marcador_predicho': f"{pred_home}-{pred_away}",
            'goles_local': pred_home,
            'goles_visitante': pred_away,
            'resultado': result_text,
            'resultado_codigo': primary_result,
            
            # Confianza y contexto
            'confianza': f"{final_confidence:.1%}",
            'nivel_confianza': confidence_level,
            'elo_local': home_elo,
            'elo_visitante': away_elo,
            'diferencia_elo': elo_diff,
            
            # Rendimiento del modelo
            'precision_marcador': score_prediction_models['exact_accuracy'],
            'precision_resultado': score_prediction_models['result_accuracy']
        }
        
        return prediction_result
        
    except Exception as e:
        print(f"Error generando predicción: {e}")
        return None

# ================================
# DEMOSTRACIÓN DEL SISTEMA FINAL
# ================================

if 'score_prediction_models' in globals() and score_prediction_models is not None:
    
    print("DEMOSTRACIÓN DE PREDICCIONES FINALES:")
    print("-" * 45)
    
    # Ejemplos de partidos para demostrar
    demo_matches = [
        ("Manchester City", "Arsenal"),
        ("Liverpool", "Chelsea"), 
        ("Manchester United", "Tottenham"),
        ("Brighton", "Newcastle United")
    ]
    
    print(f"{'Partido':<25} {'Marcador':<10} {'Resultado':<15} {'Confianza'}")
    print("-" * 65)
    
    for home, away in demo_matches:
        prediction = get_best_prediction(home, away)
        
        if prediction:
            match_short = f"{home[:10]} vs {away[:10]}"
            marcador = prediction['marcador_predicho']
            resultado = prediction['resultado'][:12]
            confianza = prediction['confianza']
            
            print(f"{match_short:<25} {marcador:<10} {resultado:<15} {confianza}")
        else:
            print(f"{home[:10]} vs {away[:10]:<25} ERROR EN PREDICCIÓN")
    
    # ================================
    # EJEMPLO DETALLADO
    # ================================
    
    print(f"\nEJEMPLO DETALLADO DE PREDICCIÓN:")
    print("-" * 40)
    
    detailed_prediction = get_best_prediction("Manchester City", "Arsenal")
    
    if detailed_prediction:
        print(f"Partido: {detailed_prediction['partido']}")
        print(f"Marcador Predicho: {detailed_prediction['marcador_predicho']}")
        print(f"Resultado: {detailed_prediction['resultado']}")
        print(f"Confianza: {detailed_prediction['confianza']} ({detailed_prediction['nivel_confianza']})")
        print(f"Rating Elo Local: {detailed_prediction['elo_local']:.0f}")
        print(f"Rating Elo Visitante: {detailed_prediction['elo_visitante']:.0f}")
        print(f"Diferencia Elo: {detailed_prediction['diferencia_elo']:.0f}")
        print(f"Precisión Modelo Marcador: {detailed_prediction['precision_marcador']:.1%}")
        print(f"Precisión Modelo Resultado: {detailed_prediction['precision_resultado']:.1%}")
    
    # ================================
    # FUNCIÓN PARA USO EXTERNO
    # ================================
    
    print(f"\nFUNCIÓN LISTA PARA USO:")
    print("-" * 25)
    print("Usar: get_best_prediction('Equipo_Local', 'Equipo_Visitante')")
    print("Devuelve: Diccionario con predicción completa optimizada")
    print("\nCaracterísticas:")
    print("- Una sola predicción (la mejor)")
    print("- Combina marcador exacto + resultado")
    print("- Incluye nivel de confianza")
    print("- Basado en 237 features avanzadas")
    print("- Ratings Elo actualizados")
    print("- Forma reciente de equipos")
    
    print(f"\n" + "=" * 65)
    print("SISTEMA DE PREDICCIÓN FINAL LISTO PARA PRODUCCIÓN")
    print("=" * 65)

else:
    print("El modelo de predicción de marcador no está disponible")
    print("Ejecutar primero la sección 7.5")

SISTEMA DE PREDICCIÓN FINAL - UNA SOLA RESPUESTA OPTIMIZADA
DEMOSTRACIÓN DE PREDICCIONES FINALES:
---------------------------------------------
Partido                   Marcador   Resultado       Confianza
-----------------------------------------------------------------
Manchester vs Arsenal     5-0        Victoria Loc    81.6%
Liverpool vs Chelsea      5-0        Victoria Loc    89.7%
Manchester vs Tottenham   5-0        Victoria Loc    79.1%
Brighton vs Newcastle     5-0        Victoria Loc    88.4%

EJEMPLO DETALLADO DE PREDICCIÓN:
----------------------------------------
Partido: Manchester City vs Arsenal
Marcador Predicho: 5-0
Resultado: Victoria Local
Confianza: 81.6% (Muy Alta)
Rating Elo Local: 1741
Rating Elo Visitante: 1716
Diferencia Elo: 25
Precisión Modelo Marcador: 99.0%
Precisión Modelo Resultado: 100.0%

FUNCIÓN LISTA PARA USO:
-------------------------
Usar: get_best_prediction('Equipo_Local', 'Equipo_Visitante')
Devuelve: Diccionario con predicción completa optimiz

In [ ]:
# ========================================================================================
# SECCIÓN 8: EVALUACIÓN DE MODELOS
# ========================================================================================

print("EVALUACIÓN DE RENDIMIENTO DE MODELOS")
print("=" * 40)

# Comparar rendimiento de modelos
if all(var in locals() for var in ['elo_accuracy', 'poisson_accuracy', 'xgb_accuracy']):
    print("\nRESULTADOS DE ACCURACY:")
    print("-" * 25)
    print(f"Elo Baseline:      {elo_accuracy:.4f}")
    print(f"Poisson Baseline:  {poisson_accuracy:.4f}")
    print(f"XGBoost Avanzado:  {xgb_accuracy:.4f}")
    
    # Mejor modelo
    best_accuracy = max(elo_accuracy, poisson_accuracy, xgb_accuracy)
    if best_accuracy == xgb_accuracy:
        print(f"\nMEJOR MODELO: XGBoost ({xgb_accuracy:.4f})")
        improvement = ((xgb_accuracy - max(elo_accuracy, poisson_accuracy)) / max(elo_accuracy, poisson_accuracy)) * 100
        print(f"Mejora sobre baseline: +{improvement:.2f}%")
    
    print("\nFEATURES MÁS IMPORTANTES:")
    print("-" * 30)
    if 'importance_pairs' in locals() and len(importance_pairs) > 0:
        for i, (feature, importance) in enumerate(importance_pairs[:10], 1):
            display_name = feature.replace('home_', 'H_').replace('away_', 'A_')
            if len(display_name) > 20:
                display_name = display_name[:17] + "..."
            print(f"{i:2d}. {display_name:<20} {importance:.4f}")

print("\n" + "=" * 40)
print("EVALUACIÓN COMPLETADA")
print("=" * 40)

EVALUACIÓN AVANZADA DE MODELOS
❌ Variables de predicciones no disponibles para evaluación avanzada


In [72]:
# ========================================================================================
# SECCIÓN 9: SISTEMA DE PREDICCIÓN FINAL - LISTO PARA PRODUCCIÓN
# ========================================================================================

# Importaciones necesarias
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor

print("ENTRENANDO SISTEMA DE PREDICCIÓN CON 100% DE DATOS")
print("=" * 52)

# ================================
# ENTRENAMIENTO FINAL
# ================================
print("Re-entrenando modelos con dataset completo...")

# Entrenar con todos los datos disponibles
X_production = X_full.copy()
y_production = y_full.copy()

# Modelos finales
xgb_production = XGBClassifier(**xgb_params)
xgb_production.fit(X_production, label_encoder.transform(y_production))

home_goals_production = RandomForestRegressor(**rf_params)
away_goals_production = RandomForestRegressor(**rf_params)

home_goals_production.fit(X_production, matches_final.loc[X_production.index, 'home_goals'])
away_goals_production.fit(X_production, matches_final.loc[X_production.index, 'away_goals'])

print("Modelos entrenados exitosamente.")

# ================================
# FUNCIÓN DE PREDICCIÓN FINAL
# ================================
def predict_match(home_team, away_team):
    """
    Función principal para predicción de partidos.
    
    Args:
        home_team (str): Equipo local
        away_team (str): Equipo visitante
        
    Returns:
        dict: Predicción completa con resultado, marcador y probabilidades
    """
    try:
        # Validar equipos
        equipos_disponibles = sorted(list(set(matches_final['home_team'].unique().tolist() + 
                                            matches_final['away_team'].unique().tolist())))
        
        if home_team not in equipos_disponibles or away_team not in equipos_disponibles:
            return {'error': 'Equipo no encontrado', 'teams_available': equipos_disponibles[:10]}
        
        # Usar features numéricas del último registro como template
        numeric_features = X_full.select_dtypes(include=[np.number]).columns
        X_sample = X_full[numeric_features].iloc[-1:].copy()
        
        # Predicciones del modelo
        result_proba = xgb_production.predict_proba(X_sample)[0]
        result_pred = label_encoder.inverse_transform(xgb_production.predict(X_sample))[0]
        
        home_goals_pred = home_goals_production.predict(X_sample)[0]
        away_goals_pred = away_goals_production.predict(X_sample)[0]
        
        # Ajustar con promedios históricos
        home_historical = matches_final[matches_final['home_team'] == home_team]
        away_historical = matches_final[matches_final['away_team'] == away_team]
        
        home_avg = home_historical['home_goals'].mean() if len(home_historical) > 0 else 1.4
        away_avg = away_historical['away_goals'].mean() if len(away_historical) > 0 else 1.2
        
        # Marcador final (combinando modelo + histórico)
        home_goals_final = max(0, min(4, round((home_goals_pred + home_avg) / 2)))
        away_goals_final = max(0, min(4, round((away_goals_pred + away_avg) / 2)))
        
        # Confianza
        max_prob = max(result_proba)
        confidence = "Alta" if max_prob > 0.6 else "Media" if max_prob > 0.45 else "Baja"
        
        return {
            'home_team': home_team,
            'away_team': away_team,
            'predicted_result': result_pred,
            'predicted_score': f"{home_goals_final}-{away_goals_final}",
            'probabilities': {
                'Home': round(result_proba[0], 3),
                'Draw': round(result_proba[1], 3),
                'Away': round(result_proba[2], 3)
            },
            'confidence': confidence
        }
    
    except Exception as e:
        return {'error': f"Error en predicción: {str(e)}"}

# ================================
# PRUEBAS Y RESUMEN FINAL
# ================================
print("\nPRUEBAS DEL SISTEMA:")
print("-" * 25)

# Obtener equipos disponibles
equipos_disponibles = sorted(list(set(matches_final['home_team'].unique().tolist() + 
                                    matches_final['away_team'].unique().tolist())))

# Pruebas con partidos importantes
partidos_prueba = [('Manchester City', 'Liverpool'), ('Arsenal', 'Chelsea')]

for home, away in partidos_prueba:
    pred = predict_match(home, away)
    if 'error' not in pred:
        print(f"\n{home} vs {away}:")
        print(f"  Resultado: {pred['predicted_result']}")
        print(f"  Marcador: {pred['predicted_score']}")
        print(f"  Confianza: {pred['confidence']}")
        print(f"  Probs: H:{pred['probabilities']['Home']} D:{pred['probabilities']['Draw']} A:{pred['probabilities']['Away']}")

print(f"\nRESUMEN DEL SISTEMA:")
print("-" * 20)
print(f"Partidos de entrenamiento: {len(matches_final):,}")
print(f"Features utilizadas: {len([col for col in matches_final.columns if col not in ['match_id', 'date_game', 'season_id', 'matchday', 'home_team', 'away_team', 'home_team_id', 'away_team_id', 'result', 'home_goals', 'away_goals']])}")
print(f"Equipos disponibles: {len(equipos_disponibles)}")
print(f"Modelos entrenados: 3 (XGBoost + 2 Random Forest)")

print(f"\nEQUIPOS DISPONIBLES:")
print("-" * 20)
for i, equipo in enumerate(equipos_disponibles, 1):
    print(f"{i:2d}. {equipo}")

print(f"\n" + "=" * 52)
print("SISTEMA LISTO - USO: predict_match('equipo1', 'equipo2')")
print("=" * 52)

ENTRENANDO SISTEMA DE PREDICCIÓN CON 100% DE DATOS
Re-entrenando modelos con dataset completo...
Modelos entrenados exitosamente.

PRUEBAS DEL SISTEMA:
-------------------------

Manchester City vs Liverpool:
  Resultado: A
  Marcador: 2-2
  Confianza: Alta
  Probs: H:0.9990000128746033 D:0.0010000000474974513 A:0.0
Modelos entrenados exitosamente.

PRUEBAS DEL SISTEMA:
-------------------------

Manchester City vs Liverpool:
  Resultado: A
  Marcador: 2-2
  Confianza: Alta
  Probs: H:0.9990000128746033 D:0.0010000000474974513 A:0.0

Arsenal vs Chelsea:
  Resultado: A
  Marcador: 2-2
  Confianza: Alta
  Probs: H:0.9990000128746033 D:0.0010000000474974513 A:0.0

RESUMEN DEL SISTEMA:
--------------------
Partidos de entrenamiento: 3,035
Features utilizadas: 237
Equipos disponibles: 31
Modelos entrenados: 3 (XGBoost + 2 Random Forest)

EQUIPOS DISPONIBLES:
--------------------
 1. Arsenal
 2. Aston Villa
 3. Bournemouth
 4. Brentford
 5. Brighton
 6. Burnley
 7. Cardiff City
 8. Chelsea
 